In [2]:
import json
import numpy as np

from bottom_up.all import train_clf, runtime_path_wild_frontier, get_f1, pick_l2r_connected


clf, vectorizer = train_clf()

tot = 0
for pno, paths in enumerate(open("validation.paths", "r")):
    paths=json.loads(paths)
    sentence = paths["sentence"]
    predicted = runtime_path_wild_frontier(sentence, pi=pick_l2r_connected, clf=clf, vectorizer=vectorizer)
    f1s = get_f1(predicted, sentence)
    tot += f1s
print(tot/(pno + 1))

0.8576927098089177


In [4]:
%timeit runtime_path_wild_frontier(sentence, pi=pick_l2r_connected, clf=clf, vectorizer=vectorizer)

5.96 ms ± 399 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [186]:

### this is just debugging code

confusion = []
from collections import Counter
from collections import defaultdict
all_vals = defaultdict(list)
scr = clf.predict(X_train)
o = 0
h = 0
tot = []
big_list = []
big_list_sentences = []

for paths in training_paths:
    paths=json.loads(paths)
    s = paths["sentence"]
    for p in paths["paths"]:
        current_tree, vertex, decision = p
        if vertex != 0:  
            big_list.append(p)
            big_list_sentences.append(s)
        

for ino, i in enumerate(train_features):
    if 'dep' in i and i["dep"] == "amod":
        o += 1
        if scr[ino] == y_train[ino]:
            h += 1
        else:
            tot.append((scr[ino], y_train[ino], big_list_sentences[ino],train_features[ino], big_list[ino]))
print(h/(len(tot) + h))
print(len(tot))


hits = 0
govs = defaultdict(int)
import string
punct = [_ for _ in string.punctuation]
alls= []
for t in tot:
    pred, was, s, feats, path = t
    current_tree, vertex, decision = path
    current_tree.sort()
    governed = [_["dep"] for _ in s["basicDependencies"] if _["dependent"] == vertex]
    for g in governed:
        govs[g] += 1
    if(pred == 0 and was == 1):
        print(feats,pred,was)
        
        
Counter(alls).most_common()
#print(hits)
#print(len(tot))
#print(sum(govs.values()))

0.7480967917346384
1853
